In [1]:
import re
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep

In [4]:
def get_driver():
    url = 'https://www.kinopoisk.ru/lists/navigator/?page=100&sort=popularity&quick_filters=available_online&tab=online'
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    dr = webdriver.Chrome(options=chrome_options)
    return dr
# dr.get(url)
# soup = BeautifulSoup(dr.page_source)
# dr.quit()

In [5]:
def get_row(data):
    name = data.find_all('p', {'class' : 'selection-film-item-meta__name'}, limit = 1)
    if (len(name) > 0): name = name[0].text;
    else: print(1); return 0
    
    link = data.find_all('a', {'class' : 'selection-film-item-meta__link'}, limit = 1)
    if (len(link) > 0): link = 'https://www.kinopoisk.ru' + link[0].get('href') 
    else: print(2); return 0
    
    date = data.find_all('p', {'class' : 'selection-film-item-meta__original-name'}, limit = 1)
    if (len(date) > 0): date = date[0].text 
    else: print(3); return 0

    date = re.search(r'[\d]{4}([-–]([\d]{4})?)?$', date)
    if (date != None): date = date.group(0) 
    else: print(4); return 0
    
    x = data.find_all('span', {'class' : 'selection-film-item-meta__meta-additional-item'}, limit = 2)
    if (len(x) > 1): country = x[0].text
    else: country = '-'
    if (len(x) > 0): genre = x[len(x) - 1].text
    else: genre = '-'
    
    rating = data.find_all('span', {'class' : r'rating__value rating__value_positive'}, limit = 1)
    if (len(rating) == 0): rating = data.find_all('span', {'class' : r'rating__value rating__value_neutral'}, limit = 1)
    if (len(rating) > 0): rating = rating[0].text 
    else: print(6); return -1
    
    rating_count = data.find_all('span', {'class' : 'rating__count'}, limit = 1)
    if (len(rating_count) > 0): rating_count = int(rating_count[0].text.replace(' ', ''))
    else: print(7); return 0
    
    subscription = data.find_all('a', {'class' : "HaRNLxUnsJBznTziWnJLc _2uafN1nqLS4lOOw1XcCR1F _2Rr4W29sDB8Q8CY5WPzzN3 _3J1Gj9qhiOSA9zzTzU0k5W"})
    if (len(subscription) > 0): 
        subscription = subscription[0].text
    else: subscription = 'online'

    if len(date) == 4: kind = 'film'
    else: kind = 'serial'
    
    return [name, date, country, genre, rating, rating_count, kind, link, subscription, 0, 0, 0]

In [6]:
def get_films():
    df = pd.DataFrame(columns=['name','date','country','genre','rating',\
'rating_count', 'kind', 'link', 'subscription', 'audio', 'subtitles', 'description'])
    url = 'https://www.kinopoisk.ru/lists/navigator/?page={}&sort=popularity&quick_filters=available_online&tab=online'
    dr = get_driver()
    i = 1
    wrong_count = 0
    while i < 200:
        if i % 20 == 0:
            print(i)
        dr.get(url.format(i))
        soup = BeautifulSoup(dr.page_source)
        data = soup.find_all('div', {'class' :'desktop-seo-selection-film-item__content-wrapper'})
        if len(data) == 0:
            print(f"i = {i}; wrong_count = {wrong_count};  wrong page")
            wrong_count += 1
            if wrong_count > 5:
                return(df)
            sleep(10)
            continue
        for j in range(len(data)):
            row = get_row(data[j])
            if row[4] < '5.5':
                print(f"i = {i}; j = {j}; bad film({row[0]}; {row[4]}), search end")
                return df
            elif (type(row) == list):
                df.loc[len(df.index)] = row
            else:
                print(f"i = {i}; j = {j}; film error")
                return df
        i += 1
    return df

In [200]:
%%time
df = get_films()
df.shape[0]

20
40
60
80
100
120
140
160
i = 168; j = 28; bad film(Каменная башка; 5.4), search end
CPU times: user 4min 4s, sys: 2.74 s, total: 4min 7s
Wall time: 25min 5s


8378

In [202]:
df.to_csv('df_films_copy.csv', index=False)

In [7]:
df = pd.read_csv("df_films.csv")
condition = (df['kind'] == 'serial')
condition &= df['country'].str.contains('Россия') == False
df = df[condition].sort_values(by='rating_count', ascending=False)
df = df.iloc[0:10]
df

,name,date,country,genre,rating,rating_count,kind,link,subscription,audio,subtitles,description
10,Игра престолов,2011–2019,"США, Великобритания","фэнтези, драма",9.0,653527,serial,https://www.kinopoisk.ru/film/464963/,По подписке Плюс Мульти с Амедиатекой,0,0,0
8,Рик и Морти,2013–,США,"мультфильм, комедия",9.0,634209,serial,https://www.kinopoisk.ru/film/685246/,По подписке Плюс,0,0,0
47,"Ну, погоди!",1969–2017,СССР,"мультфильм, комедия",8.7,585054,serial,https://www.kinopoisk.ru/film/46483/,По подписке Плюс,0,0,0
0,Друзья,1994–2004,США,"комедия, мелодрама",9.2,420108,serial,https://www.kinopoisk.ru/film/77044/,По подписке Плюс,0,0,0
23,Во все тяжкие,2008–2013,США,"криминал, драма",8.9,397320,serial,https://www.kinopoisk.ru/film/404900/,online,0,0,0
30,Доктор Хаус,2004–2012,США,"драма, детектив",8.8,368965,serial,https://www.kinopoisk.ru/film/178710/,По подписке Плюс,0,0,0
81,Теория большого взрыва,2007–2019,США,"комедия, мелодрама",8.6,353011,serial,https://www.kinopoisk.ru/film/306084/,По подписке Плюс,0,0,0
272,Сверхъестественное,2005–2020,"США, Канада","фэнтези, ужасы",8.2,351137,serial,https://www.kinopoisk.ru/film/178707/,По подписке Плюс,0,0,0
12,Гравити Фолз,2012–2016,США,"мультфильм, комедия",9.0,337385,serial,https://www.kinopoisk.ru/film/591929/,По подписке Плюс,0,0,0
731,Ходячие мертвецы,2010–2022,США,"ужасы, триллер",8.0,336963,serial,https://www.kinopoisk.ru/film/508161/,По подписке Плюс,0,0,0


In [145]:
'aaa' in 'aaaab'

True

In [8]:
def get_add_info(df, name):
    if name not in list(df1.name):
        return('Something wrong')
    index = df.loc[(df.name == name)].index[0]
    if str(df.iloc[index]['audio']) != '0':
        return df
    link = df.iloc[index]['link']
    dr = webdriver.Chrome()
    try:
        dr.get(link)
    except:
        return('Something wrong')
    soup1 = BeautifulSoup(dr.page_source)
    info = soup1.find_all('div', {'class' : 'styles_valueLight__1j0RO'}, limit = 2)
    if (len(info) > 1): audio_track = info[0].text; subtitles = info[1].text
    else: return('Something wrong')
    description = soup1.find_all('p', {'class' : 'styles_paragraph__2Otvx'}, limit = 1)
    if (len(description) > 0): description = description[0].text
    else: return('Something wrong')
    
    df.iloc[index]['audio'] = audio_track
    df.iloc[index]['subtitles'] = subtitles
    df.iloc[index]['description'] = description
    return df


def add_info_to_str(df, name):
    index = df.loc[(df.name == name)].index[0]
    elem = df.iloc[index]
    country = elem['country']
    date = elem['date']
    audio_track = elem['audio']
    subtitles = elem['subtitles']
    description = elem['description']
    s = name + '\\n\\n'
    s = 'Страна:\\n' + country + '\\n\\n'
    s += 'Год:\\n' + date + '\\n\\n'
    s += 'Язык озвучки:\\n' + audio_track + '\\n\\n'
    s += 'Субтитры:\\n' + subtitles + '\\n\\n'
    s += 'Описание:\\n' + description
    return s

In [62]:
#data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
genre = ['', 'anime', 'mystery', 'comedy', 'crime', 'adventure', 'family', 'thriller', 'horror', 'sci-fi', 'fantasy']
sort = ['', 'popularity', 'year']
rating = ['', 'high_rated']
subscript= ['available_online', 'yandex_plus_subscription', 'kinopoisk_with_amediateka_subscription']
country = ['', 'foreign', 'russian']
kind = ['', 'films', 'serials']

In [18]:
def df_to_str(df):
    df = df.set_index('name')
    s = ''
    for i in range(len(df)):
        name = df.index[i]
        s += '<b>' + str(name) + '</b>\\n\\n'
        s += 'Рейтинг' + ' ' * 17 + '-' + ' ' * 5 + str(df.loc[name]['rating']) + '\\n'
        s += 'Кол-во оценок     ' + '-' + ' ' * 5 + str(df.loc[name]['rating_count']) + '\\n'
        genre_list = str(df.loc[name]['genre']).split(', ')
        s += 'Жанр' + ' ' * 22 + '-' + ' ' * 5 + genre_list[0]
        if len(genre_list) == 2:
            s += '\\n' + ' ' * 39 + genre_list[1]
        s += '\\n'
        if i != len(df) - 1:
            s += '\\n\\n\\n'
    return(s)
s = df_to_str(d1)
#print(s)

In [9]:
df1 = d1
df2 = d2
#df1.to_csv('csv_test.csv', index=False)
df1 = pd.read_csv("csv_test.csv")
#s = df_to_str(df1)
#print(df1)
#df1 = df1.set_index('name')
#df1 = df1[df1['name'].isin(df2['name']) == True]
#df1 = df1.drop(index)
# print('\n\n')
print(type(df1.iloc[1]['audio']))
df1['audio'] = 'ss'
print(type(df1.iloc[1]['audio']))
print(df1)

<class 'numpy.int64'>
<class 'str'>
                  name  date                  country               genre  \
0              Легенда  2015  Великобритания, Франция   криминал, триллер   
1  Алита: Боевой ангел  2019              США, Япония  фантастика, боевик   
2                Довод  2020      Великобритания, США  фантастика, боевик   

   rating rating_count                                    link audio  \
0     7.2      387 977   https://www.kinopoisk.ru/film/839954/    ss   
1     7.1      384 969    https://www.kinopoisk.ru/film/88173/    ss   
2     7.6      384 587  https://www.kinopoisk.ru/film/1236063/    ss   

   subtitles  description  
0          0            0  
1          0            0  
2          0            0  
